In [ ]:
import altair as alt
import polars as pl

%load_ext autoreload
%autoreload 2

# Base Load analysis

Demo of a base load analysis for a dossier, we define some KPIs we want to measure.



## loading in the data

# Base Load Analysis Demo

This notebook demonstrates how to analyze base load (standby power consumption) in energy usage data. Base load represents the minimum continuous power draw in a system, typically from devices that are always on or in standby mode.

## Key Metrics

We analyze three core metrics:
1. Base load value in WATTS - Shows the consistent minimum power draw
2. Energy consumption in kWh - Quantifies power used over time
3. Base load percentage - Shows what portion of total consumption is baseline

## Data Format Requirements

The analysis expects data in the following format:
- Timestamp (datetime with timezone 'Europe/Brussels')
- Total power (float, in kW)

Example input data structure:
```json
{
    "timestamp": "2024-01-01T00:00:00+01:00",
    "total": 0.5
}


> ⚠️**Note:** we have several example files available,  
> *energy_use_big* is from a giant building with incredible base load.  
> *energy_use_test1* is from a regular family residence.


In [ ]:
from openenergyid.baseload.main import load_energy_data, analyze_base_load
from openenergyid.enums import Granularity

# example_file = "data/PP/energy_use_big.ndjson"
example_file = "data/PP/energy_use_test1.ndjson"
# Load energy data
energy_data = load_energy_data(example_file)

# Monthly analysis
monthly_metrics = analyze_base_load(energy_data, Granularity.P1M)

# Daily analysis
daily_metrics = analyze_base_load(energy_data, Granularity.P1D)

# Hourly analysis
hourly_metrics = analyze_base_load(energy_data, Granularity.PT1H)

display(monthly_metrics)
display(daily_metrics)
display(hourly_metrics)

In [ ]:
def create_monthly_chart(df):
    """Create bar chart for monthly data"""
    bars = (
        alt.Chart(df)
        .mark_bar(opacity=0.7)
        .encode(
            x=alt.X("period_start:T", timeUnit="month", title="Month"),
            y=alt.Y("total_usage:Q", title="Total Usage (kWh)"),
            tooltip=["period_start:T", "total_usage:Q", "base_percentage:Q"],
        )
    )

    base_line = (
        alt.Chart(df)
        .mark_bar(color="purple", opacity=0.5)
        .encode(
            x=alt.X("period_start:T", timeUnit="month"),
            y=alt.Y("base_load_kwh:Q"),
            tooltip=["base_load_kwh:Q", "base_percentage:Q"],
        )
    )

    return (bars + base_line).properties(
        width=800, height=200, title="Monthly Energy Usage and Base Load"
    )


def create_daily_chart(df):
    """Create line chart for daily data"""
    lines = (
        alt.Chart(df)
        .mark_line(opacity=0.7)
        .encode(
            x=alt.X("period_start:T", title="Date"),
            y=alt.Y("total_usage:Q", title="Daily Usage (kWh)"),
            tooltip=["period_start:T", "total_usage:Q", "base_percentage:Q"],
        )
    )

    base = (
        alt.Chart(df)
        .mark_line(color="purple")
        .encode(
            x="period_start:T",
            y="base_load_kwh:Q",
            tooltip=["base_load_kwh:Q", "base_percentage:Q"],
        )
    )

    return (lines + base).properties(width=800, height=200, title="Daily Energy Usage Pattern")


def create_hourly_chart(df):
    """Create line chart for sampled hourly data"""
    # Sample data to prevent overcrowding
    sampled = df.sample(n=500, seed=42)

    return (
        alt.Chart(sampled)
        .mark_line(opacity=0.7)
        .encode(
            x=alt.X("period_start:T", title="Hour"),
            y=alt.Y("total_usage:Q", title="Hourly Usage (kWh)"),
            tooltip=["period_start:T", "total_usage:Q", "base_percentage:Q"],
        )
        .properties(width=800, height=200, title="Hourly Energy Usage (Sampled)")
    )


# Combine all charts
combined = alt.vconcat(
    create_monthly_chart(monthly_metrics),
    create_daily_chart(daily_metrics),
    create_hourly_chart(hourly_metrics),
).resolve_scale(y="independent")

combined.display()

# Test analysis

In [ ]:
energy_use_lf_1 = pl.scan_ndjson(
    example_file,
    schema={"timestamp": pl.Datetime(time_zone="Europe/Brussels"), "total": pl.Float64},
)
testframe = (
    energy_use_lf_1.filter(pl.col("total") >= 0).sort("total", descending=True)
    # .with_columns(pl.col("timestamp").interpolate(method="linear"))
)
tf = testframe.collect()
display(tf)
display(energy_use_lf_1.collect().head())

In [ ]:
# Compute the value counts using Polars
value_counts = tf.group_by("total").agg(pl.count("total").alias("count")).sort("total")


bar_chart = (
    alt.Chart(value_counts)
    .mark_area()
    .encode(
        x=alt.X("total:Q", title="Power Usage Value"),
        y=alt.Y("count:Q", title="Count"),
        tooltip=["total", "count"],
    )
    .properties(width=800, height=400, title="Power Usage Value Counts")
)

# Display the bar chart
bar_chart.display()

the main meat of the analysis
defining the useful columns for our data on which we will do some further analysis.
to do some useful comparisons too


In [ ]:
lf = (
    energy_data.filter(pl.col("total") >= 0)
    .sort("timestamp")
    .group_by_dynamic("timestamp", every="1d")
    .agg(
        [
            pl.sum("total").alias("total_daily_usage"),
            pl.min("total").alias("lowest_recorded"),
            (pl.min("total") * 4 * 24).alias(
                "min_power_usage_per_day"
            ),  # what if you only used the lowest recorded power usage for that day
            (pl.max("total") * 4 * 24).alias(
                "max_power_usage_per_day"
            ),  # what if you used the max power usage for that day
        ]
    )
)

df_extended = lf.collect()  # df is same file with more columns
df_extended.describe(percentiles=[0.01, 0.05, 0.1, 0.25])
display(df_extended.head())

In [ ]:
lf = (
    energy_use_lf_1.filter(pl.col("total") >= 0)
    .sort("timestamp")
    .group_by_dynamic("timestamp", every="1d")
    .agg(
        [
            pl.sum("total").alias("total_daily_usage"),
            (pl.col("total").sort().head(10).mean() * 4 * 24).alias(
                "basislast_daily_kwh"
            ),  # Mean of x lowest values
        ]
    )
)

df = lf.collect()

# Calculate overall metrics
df = df.with_columns(
    [
        (pl.col("basislast_daily_kwh") / pl.col("total_daily_usage") * 100).alias(
            "basislast_percentage"
        )
    ]
)

# Calculate averages across period
avg_basislast_daily = df["basislast_daily_kwh"].mean()
avg_basislast_watts = avg_basislast_daily * 1000 / 24  # Convert daily kWh to average watts

avg_percentage = df["basislast_percentage"].mean()

print(f"Average Basislast: {avg_basislast_watts:.1f}W")
print(f"Average Daily Usage: {avg_basislast_daily:.1f} kWh")
print(f"Average Percentage: {avg_percentage:.1f}%")

In [ ]:
total_new = (
    alt.Chart(df)
    .mark_area()
    .encode(
        x=alt.X("timestamp:T", title="Time"),
        y=alt.Y("total_daily_usage:Q", title="Energy use (kWh)"),
        tooltip=["timestamp:T", "total_daily_usage:Q"],
    )
    .properties(width=1000, height=400)
)
lowest_new = (
    alt.Chart(df)
    .mark_line(color="purple")
    .encode(
        x="timestamp:T",
        y=alt.Y("basislast_daily_kwh:Q"),
        tooltip=[
            "timestamp:T",
            alt.Tooltip("basislast_daily_kwh:Q", title="Basislast (kWh)"),
            alt.Tooltip("basislast_percentage:Q", title="% of Total", format=".1f"),
        ],
    )
)

daily_comparison = total_new + lowest_new
# daily_comparison.display()
# Draw a line at y=0.6
rule = alt.Chart(pl.DataFrame({"y": avg_basislast_daily})).mark_rule(color="purple").encode(y="y:Q")

# Combine the rule with the existing chart
(daily_comparison + rule).display()

In [ ]:
month_filter = "month(datum.timestamp) == 2"  # Altair datetime function syntax

total_new = (
    alt.Chart(df)
    .mark_area()
    .encode(
        x=alt.X("timestamp:T", title="Time"),
        y=alt.Y("total_daily_usage:Q", title="Energy use (kWh)"),
        tooltip=["timestamp:T", "total_daily_usage:Q"],
    )
    .transform_filter(month_filter)
    .properties(width=1000, height=400)
)

lowest_new = (
    alt.Chart(df)
    .mark_line(color="purple")
    .encode(
        x="timestamp:T",
        y=alt.Y("basislast_daily_kwh:Q"),
        tooltip=[
            "timestamp:T",
            alt.Tooltip("basislast_daily_kwh:Q", title="Basislast (kWh)"),
            alt.Tooltip("basislast_percentage:Q", title="% of Total", format=".1f"),
        ],
    )
    .transform_filter(month_filter)
)

rule = alt.Chart(pl.DataFrame({"y": 1})).mark_rule(color="purple").encode(y="y:Q")

(total_new + lowest_new + rule).display()

In [ ]:
# Resample to monthly totals
monthly_lf = (
    energy_use_lf_1.filter(pl.col("total") >= 0)
    .group_by_dynamic("timestamp", every="1mo")
    .agg(
        [
            pl.sum("total").alias("total_monthly_usage"),
            (pl.col("total").sort().head(10).mean() * 4 * 24 * 30).alias("basislast_monthly_kwh"),
        ]
    )
)

monthly_df = monthly_lf.collect()

# Create stacked bar chart
chart = alt.Chart(monthly_df).mark_bar().encode(
    x=alt.X("month(timestamp):O", title="Month"),
    y=alt.Y("total_monthly_usage:Q", title="Energy Usage (kWh)"),
    color=alt.value("blue"),  # Green for total
) + alt.Chart(monthly_df).mark_bar().encode(
    x="month(timestamp):O",
    y="basislast_monthly_kwh:Q",
    color=alt.value("green"),  # Light green for basislast
    tooltip=[
        alt.Tooltip("month(timestamp):T", title="Month"),
        alt.Tooltip("total_monthly_usage:Q", title="Total Usage (kWh)"),
        alt.Tooltip("basislast_monthly_kwh:Q", title="Basislast (kWh)"),
        # alt.Tooltip("basislast_percentage:Q", title="Basislast %"),
    ],
)
text = (
    alt.Chart(monthly_df)
    .mark_text(
        align="center",
        baseline="middle",
        dy=-10,  # Adjust vertical position
        color="black",
    )
    .encode(text=alt.Text("Percentage:Q", format=".2f"))
)

chart.properties(width=800, height=400, title="Monthly Energy Usage with Basislast")

In [ ]:
display(monthly_df)

This Python code uses the Polars library to process and analyze energy usage data from a NDJSON file named `energy_use2.ndjson`. The data is expected to have a timestamp and a total energy usage value.

1. **Reading Data**: The `pl.scan_ndjson` function reads the NDJSON file into a lazy frame (`energy_use_lf`) with a specified schema.
2. **Filtering and Sorting**: The data is filtered to include only non-negative `total` values and then sorted by the `timestamp`.
3. **Grouping and Aggregation**: The data is grouped dynamically by day (`every="1d"`), and several aggregations are performed:
   - Sum of `total` energy usage per day.
   - Minimum `total` energy usage per day.
   - Minimum power usage per day calculated as `min(total) * 4 * 24`.
   - Percentage of standby usage calculated as `(min(total) / sum(total)) * 100`.
4. **Collecting Results**: The results are collected into a DataFrame (`df`).

This code effectively prepares and summarizes daily energy usage statistics.

In [ ]:
alt.data_transformers.enable("vegafusion")

# Create a line chart for the testframe
line_chart = (
    alt.Chart(tf)
    .mark_line()
    .encode(x="timestamp:T", y="total:Q", tooltip=["timestamp:T", "total:Q"])
    .properties(width=800, height=400, title="Energy Usage Over Time with LOESS Smoothing")
)

# Add LOESS smoothing
loess_chart = line_chart.transform_loess("timestamp", "total", bandwidth=0.003).mark_line(
    color="red"
)

# Combine the line chart and LOESS chart
combined_chart = line_chart + loess_chart

# Display the combined chart
combined_chart.display()

In [ ]:
alt.data_transformers.enable("vegafusion")
# Calculate the xth percentile
qx = tf["total"].quantile(0.98)

# Filter the data to include only the data below the 95th percentile
filtered_tf = tf.filter(pl.col("total") <= qx)

# Create the density plot
alt.Chart(filtered_tf).transform_density(
    "total",
    as_=["total", "density"],
).mark_area().encode(
    x="total:Q",
    y="density:Q",
).properties(width=900, height=400)

The chart plots the kernel density estimate (KDE) of the `total` energy usage values, filtered to include only the data below the 98th percentile. 

### Context:
- **X-axis (`total:Q`)**: Represents the energy usage values.
- **Y-axis (`density:Q`)**: Represents the estimated probability density of the energy usage values.

### Interpretation:
- The KDE plot provides a smoothed estimate of the distribution of energy usage values.
- Peaks in the density plot indicate the most common energy usage values.
- Filtering to the 98th percentile removes extreme values (outliers), focusing on the more typical usage patterns.

This helps in understanding the distribution and common ranges of energy usage in the dataset, highlighting where most of the energy usage values lie.

In [ ]:
import altair as alt
import polars as pl

# Assuming df_extended is already defined and contains the correct columns

# Create the max chart
max_chart = (
    alt.Chart(df_extended)
    .mark_area()
    .encode(
        alt.X("timestamp:T", axis=alt.Axis(title="Time")),
        alt.Y("total_daily_usage:Q", axis=alt.Axis(title="Energy use (kWh)")),
        tooltip=[
            alt.Tooltip("timestamp:T", title="Date"),
            alt.Tooltip("total_daily_usage:Q", title="Total Daily Usage"),
        ],
    )
    .properties(width=1000, height=400)
)

# Create the lowest chart
lowest_chart = (
    alt.Chart(df_extended)
    .mark_area()
    .encode(
        alt.X("timestamp:T", axis=alt.Axis(title="Time")),
        alt.Y("min_power_usage_per_day:Q", axis=alt.Axis(title="Energy use (kWh)")),
        tooltip=[
            alt.Tooltip("timestamp:T", title="Date"),
            alt.Tooltip("min_power_usage_per_day:Q", title="Minimal Recorded Peak Usage"),
        ],
        color=alt.value("orange"),
    )
    .properties(width=1000, height=400)
)

# Combine the charts
final_chart = max_chart + lowest_chart

# Display the final chart
final_chart.display()

In [ ]:
# Clean calculation of percentage with proper handling of edge cases
df_extended = df_extended.with_columns(
    [
        # Calculate percentage only when conditions make sense
        pl.when(
            (pl.col("total_daily_usage") > 0.1)  # Meaningful total usage threshold
            & (
                pl.col("min_power_usage_per_day") < pl.col("total_daily_usage")
            )  # Logical constraint
        )
        .then((pl.col("min_power_usage_per_day") / pl.col("total_daily_usage") * 100))
        .otherwise(None)  # Use None for invalid cases
        .alias("percentage_standby_usage")
    ]
)

# Update the standby_percentage chart to handle the cleaned data
standby_percentage = (
    alt.Chart(df_extended)
    .mark_area()
    .encode(
        x="timestamp:T",
        y=alt.Y(
            "percentage_standby_usage:Q",
            axis=alt.Axis(format="%"),
            scale=alt.Scale(domain=[0, 100]),  # Force scale from 0-100%
        ),
        tooltip=[
            "timestamp:T",
            alt.Tooltip("percentage_standby_usage:Q", format=".1f", title="Standby %"),
            alt.Tooltip("total_daily_usage:Q", format=".2f", title="Total Usage"),
            alt.Tooltip("min_power_usage_per_day:Q", format=".2f", title="Min Power"),
        ],
    )
    .properties(width=800, height=300, title="Daily Standby Usage Percentage")
)

In [ ]:
import altair as alt
import polars as pl

# Assuming df_extended is already defined and contains the correct columns

# Define a threshold for very small total_daily_usage values
threshold = 0.01  # Adjust this value as needed

# Create the percentage_standby_usage column if it doesn't exist
if "percentage_standby_usage" not in df_extended.columns:
    df_extended = df_extended.with_columns(
        pl.when(pl.col("total_daily_usage") > threshold)
        .then((pl.col("min_power_usage_per_day") / pl.col("total_daily_usage") * 100))
        .otherwise(0)
        .alias("percentage_standby_usage")
    )

# 1. Standby Usage vs Total Usage Over Time
standby_vs_total = (
    alt.Chart(df_extended)
    .transform_fold(["total_daily_usage", "min_power_usage_per_day"], as_=["variable", "value"])
    .mark_line()
    .encode(
        x="timestamp:T",
        y="value:Q",
        color="variable:N",
        tooltip=["timestamp:T", "value:Q", "variable:N"],
    )
    .properties(width=800, height=400, title="Total Energy Usage vs Standby Usage Over Time")
)

# 2. Daily Standby Percentage
standby_percentage = (
    alt.Chart(df_extended)
    .mark_area()
    .encode(
        x="timestamp:T",
        y=alt.Y("percentage_standby_usage:Q", axis=alt.Axis(format="%")),
        tooltip=["timestamp:T", alt.Tooltip("percentage_standby_usage:Q", format=".2%")],
    )
    .properties(width=800, height=300, title="Daily Standby Usage Percentage")
)

# 3. Heatmap of Standby Usage
heatmap = (
    alt.Chart(df_extended)
    .mark_rect()
    .encode(
        x=alt.X("date(timestamp):O", title="Date"),
        y=alt.Y("month(timestamp):O", title="Month"),
        color=alt.Color("min_power_usage_per_day:Q", scale=alt.Scale(scheme="viridis")),
        tooltip=["timestamp:T", "min_power_usage_per_day:Q"],
    )
    .properties(width=800, height=400, title="Heatmap of Daily Standby Power Usage")
)

# 4. Distribution of Standby Usage
histogram = (
    alt.Chart(df_extended)
    .mark_bar()
    .encode(
        x=alt.X("min_power_usage_per_day:Q", bin=True),
        y="count()",
        tooltip=["min_power_usage_per_day:Q", "count()"],
    )
    .properties(width=400, height=300, title="Distribution of Daily Standby Power Usage")
)

# 5. Scatter plot of Total vs Standby Usage
scatter = (
    alt.Chart(df_extended)
    .mark_circle()
    .encode(
        x="total_daily_usage:Q",
        y="min_power_usage_per_day:Q",
        color=alt.Color("month(timestamp):N", scale=alt.Scale(scheme="category10")),
        tooltip=["timestamp:T", "total_daily_usage:Q", "min_power_usage_per_day:Q"],
    )
    .properties(width=500, height=500, title="Total Daily Usage vs Standby Usage")
)

# Combine charts
combined_chart = (standby_vs_total & standby_percentage) | (heatmap & (histogram & scatter))
# Display the combined chart
combined_chart.display()

In [ ]:
# Add seasonal decomposition to detect patterns
from statsmodels.tsa.seasonal import seasonal_decompose


def analyze_seasonality(df):
    # Resample to daily frequency and fill gaps
    daily = (
        df.select([pl.col("timestamp"), pl.col("total").alias("value")])
        .group_by_dynamic("timestamp", every="1d")
        .agg([pl.col("value").mean()])
    )

    # Decompose into trend, seasonal, residual
    result = seasonal_decompose(
        daily.sort("timestamp").get_column("value"),
        period=7,  # Weekly seasonality
    )
    return result

In [ ]:
import altair as alt
import polars as pl


def analyze_and_plot_seasonality(df: pl.LazyFrame) -> tuple[alt.VConcatChart, pl.DataFrame]:
    """
    Analyze seasonality of energy usage data and create visualization

    Args:
        df: LazyFrame with timestamp and total columns
    Returns:
        tuple containing (altair chart, results dataframe)
    """
    # Prepare daily data
    daily = (
        df.filter(pl.col("total") >= 0)
        .group_by_dynamic("timestamp", every="1d")
        .agg(pl.col("total").mean().alias("value"))
        .sort("timestamp")
        .collect()
    )

    # Get numpy array for decomposition
    values = daily.get_column("value").to_numpy()

    # Perform decomposition
    decomposition = seasonal_decompose(
        values,
        period=7,  # Weekly seasonality
        extrapolate_trend=True,
    )

    # Create results DataFrame
    result_df = pl.DataFrame(
        {
            "timestamp": daily.get_column("timestamp"),
            "observed": values,
            "trend": decomposition.trend,
            "seasonal": decomposition.seasonal,
            "residual": decomposition.resid,
        }
    )

    # Create separate charts for each component
    def create_component_chart(title: str, y_column: str) -> alt.Chart:
        return (
            alt.Chart(result_df)
            .mark_line()
            .encode(
                x=alt.X("timestamp:T", title="Date"),
                y=alt.Y(f"{y_column}:Q", title="Value"),
                tooltip=[
                    alt.Tooltip("timestamp:T", title="Date"),
                    alt.Tooltip(f"{y_column}:Q", format=".2f"),
                ],
            )
            .properties(width=800, height=150, title=title)
        )

    # Combine charts vertically
    chart = alt.vconcat(
        create_component_chart("Original Data", "observed"),
        create_component_chart("Trend Component", "trend"),
        create_component_chart("Seasonal Component", "seasonal"),
        create_component_chart("Residual Component", "residual"),
    ).resolve_scale(y="independent")

    return chart, result_df


# Usage example
chart, decomp_data = analyze_and_plot_seasonality(energy_data)
chart.display()

# Calculate insights using Polars expressions
insights = decomp_data.select(
    [
        pl.col("observed").mean().alias("daily_avg"),
        pl.col("seasonal").abs().mean().alias("seasonal_var"),
        (pl.col("trend").diff().mean() > 0).alias("is_increasing"),
    ]
).row(0)

print("\nKey Insights:")
print(f"Average daily consumption: {insights[0]:.2f} kWh")
print(f"Seasonal variation range: {insights[1]:.2f} kWh")
print(f"Trend direction: {'Increasing' if insights[2] else 'Decreasing'}")

# Some notes about the difference between the large building and the small residential building

perhaps some future optimisations for B2B clients.

## Comparing Residential vs Commercial Energy Patterns

### Large Building Analysis
- High baseline consumption (41.57 kWh daily average)
- Strong weekly cycles (±8.16 kWh variations)
- Gradual decrease in consumption trend over the year
- Clear business hours pattern visible in seasonal component
- Several significant drops visible in residuals (possibly holidays/closures)

### Key Differences from Residential Pattern
| Aspect | Residential | Commercial Building |
|--------|-------------|-------------------|
| Daily Average | 0.10 kWh | 41.57 kWh |
| Seasonal Impact | ±0.01 kWh | ±8.16 kWh |
| Trend | Increasing | Decreasing |
| Pattern | Less structured | Strong weekly rhythm |

### Valuable Insights
1. **Operational Efficiency**
  - Commercial building shows improving efficiency (decreasing trend)
  - Residential consumption gradually increasing, might need attention

2. **Usage Patterns**
  - Commercial: Very predictable weekly cycles
  - Residential: More irregular, lifestyle-dependent

3. **Optimization Opportunities**
  - Commercial: Focus on reducing weekend/night consumption
  - Residential: Address growing base load trend

4. **Anomaly Detection**
  - Commercial building shows clear operational vs non-operational periods
  - Makes it easier to spot inefficiencies or system issues

## Seasonal Decomposition Analysis for RESIDENTIAL example1

The time series analysis of energy consumption reveals several interesting patterns:

### Overall Consumption Pattern
- Average daily consumption is relatively low at 0.10 kWh
- Shows consistent base load with regular peaks throughout the year
- Notable higher consumption periods in winter months (November-December)

### Trend Analysis
- Overall increasing trend throughout 2023
- Noticeable dip in August (possibly vacation period)
- Stronger upward trend in the latter part of the year
- Base consumption has gradually increased from ~0.05 kWh to ~0.15 kWh

### Seasonal Patterns
- Clear weekly cycles in consumption (±0.01 kWh variation)
- Weekend/weekday patterns are visible in the seasonal component
- Relatively stable seasonal pattern suggests consistent weekly routines

### Unusual Events (Residuals)
- Several significant spikes above normal usage
- August shows lower variability, supporting the vacation hypothesis
- December shows increased volatility, possibly due to holiday activities

This analysis suggests a household with very efficient base load but gradually increasing consumption over the year. The weekly patterns are highly regular, indicating consistent lifestyle habits. The increasing trend might warrant attention to prevent further base load growth.

#  General Analysis Remarks and Future Improvements

## Key Visualization Insights
1. **Most Effective Client Visualizations**:
   - Monthly bar chart (base load vs total usage) - Most intuitive for clients
   - Daily pattern visualization - Helps identify anomalies
   - Simple cost overlays - Translates energy into financial impact
   
2. **Advanced Visualizations** (for expert analysis):
   - Heatmap view - Reveals seasonal/weekly patterns
   - Distribution plots - Shows consumption patterns
   - Load curve analysis - Technical energy analysis

## Potential Analysis Enhancements

### Time Series Analysis
- Implement seasonal decomposition to detect patterns
- Add change point detection for base load shifts
- Correlate with weather data
- Analyze weekend vs weekday patterns

### Smart KPIs and Benchmarks
1. **Comparative Metrics**:
   - Base load vs similar buildings/households
   - Historical improvement tracking
   - Seasonal adjustments

2. **Financial Insights**:
   - Yearly standby power cost
   - Potential savings calculations
   - Return on investment for improvements

### Edge Cases and Data Quality
- Vacation periods and very low usage days
- Grid outages and meter issues
- Solar PV impact handling
- Seasonal base load variations

## Recommended Client Features

### Actionable Insights
1. Simple traffic light system for base load performance
2. Concrete energy-saving recommendations
3. Clear cost implications of current base load
4. Easy comparison between different time periods

### Smart Analytics
- Device-level consumption estimates
- Anomaly detection and alerts
- Predictive base load trends
- Custom benchmarking

# Next Steps

1. Implement basic benchmarking system
2. Add financial impact calculations
3. Develop anomaly detection
4. Create automated recommendations
5. Add weather correlation analysis
6. Implement device-level disaggregation
